In [17]:
import sys
sys.path.insert(0, "/home/agueven/SDV-ML")
import torch
from torchinfo import summary
import glob
from utils.vtxLevelDataset3 import ModifiedUprootIterator
from functools import partial
import user_scripts.preprocess as preprocess

MODEL_PATH = "/scratch/agueven/ParT_saved_models/vtx_PART-419best_valacc_epoch.pt"
model = torch.load(MODEL_PATH, map_location=torch.device('cuda'))


branchDict = {}
branchDict['ev'] = ['MET_phi',
                    'nSDVSecVtx',
                    ]

branchDict['sv'] = ['SDVSecVtx_pt', 
                    'SDVSecVtx_pAngle', 
                    'SDVSecVtx_charge', 
                    'SDVSecVtx_ndof', 
                    'SDVSecVtx_chi2', 
                    'SDVSecVtx_tracksSize', 
                    'SDVSecVtx_sum_tkW', 
                    'SDVSecVtx_LxySig', 
                    'SDVSecVtx_L_phi', 
                    'SDVSecVtx_L_eta', 
                    ]

branchDict['tk'] = ['SDVTrack_pt', 'SDVTrack_ptError', 
                    'SDVTrack_eta', 'SDVTrack_etaError',
                    'SDVTrack_phi', 'SDVTrack_phiError',
                    'SDVTrack_dxy', 'SDVTrack_dxyError', 
                    'SDVTrack_dz', 'SDVTrack_dzError',
                    'SDVTrack_normalizedChi2'
                    ]

branchDict['lut'] = ['SDVIdxLUT_SecVtxIdx', 
                     'SDVIdxLUT_TrackIdx',
                     'SDVIdxLUT_TrackWeight'
                     ]

branchDict['jet'] = ['Jet_phi', 
                     'Jet_eta',
                     'Jet_jetId',
                     'Jet_pt',
                     'Jet_chEmEF',
                     'Jet_neEmEF',
                     'Jet_muonIdx1',
                     'Jet_muonIdx2'
                     ]



branchDict['label'] = ['SDVSecVtx_matchedLLPnDau_bydau']

step_size = 3000
nWorkers=1

INPUTDIR = "/scratch/agueven/Ang_GNN_nano_merged/qcdht0700_2018"
files = glob.glob(f'{INPUTDIR}/**/*.root', recursive=True)
for file in files: testDict = {'sig': [file + ':Events'],'bkg': []}

preprocess_fn = partial(preprocess.transform, branch_dict=branchDict)

testDataset = ModifiedUprootIterator(testDict, 
                                     branchDict,
                                     shuffle=False,
                                     nWorkers=nWorkers,
                                     step_size=step_size)

testLoader = torch.utils.data.DataLoader(testDataset,
                                         num_workers=nWorkers,
                                         prefetch_factor=1,
                                         persistent_workers= True,
                                         collate_fn=preprocess_fn,
                                         pin_memory=True)

for batch_num, X in enumerate(testLoader):
    for key in X.keys():
        print(key + ': ', X[key].shape)
    break

Initialize iterable dataset
nWorkers:  1


/tmp/ipykernel_1878930/3903634756.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(MODEL_PATH, map_location=torch.device('cuda'))


__iter__ is called.
tk_pair_features:  torch.Size([4619, 4, 10])
tk_features:  torch.Size([4619, 11, 10])
sv_features:  torch.Size([4619, 11, 1])
label:  torch.Size([4619, 1])


In [24]:
MODEL_PATH = "/scratch/agueven/ParT_saved_models/vtx_PART-419best_valacc_epoch.pt"
model = torch.load(MODEL_PATH, map_location=torch.device('cpu'))


summary(model.module,
        input_data={'x':X['tk_features'].cpu(), 'v':X['tk_pair_features'].cpu(), 'x_sv':X["sv_features"].cpu()},
        depth=10,
        col_names=("input_size","output_size","num_params","kernel_size","mult_adds"),
       )

/tmp/ipykernel_1878930/4194729012.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(MODEL_PATH, map_location=torch.device('cpu'))


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
ParticleTransformerDVTagger                   --                        [4619, 2]                 128                       --                        --
├─SequenceTrimmer: 1-1                        [4619, 11, 10]            [4619, 11, 10]            --                        --                        --
├─Embed: 1-2                                  [4619, 11, 10]            [10, 4619, 128]           --                        --                        --
│    └─BatchNorm1d: 2-1                       [4619, 11, 10]            [4619, 11, 10]            22                        --                        101,618
│    └─Sequential: 2-2                        [10, 4619, 11]            [10, 4619, 128]           --                        --                        --
│    │    └─LayerNorm: 3-1                    [10, 4619, 11]          

In [12]:
for k,v in X.items():
    print(v.shape)

torch.Size([4619, 4, 10])
torch.Size([4619, 11, 10])
torch.Size([4619, 11, 1])
torch.Size([4619, 1])
